In [7]:
import numpy as np
import vtk
import matplotlib.pyplot as plt
import vtkplotlib as vpl
from vtk.util.numpy_support import vtk_to_numpy
import os
from scipy import spatial

In [8]:
def readfile(filename):
    reader = vtk.vtkGenericDataObjectReader()
    reader.SetFileName(filename)
    reader.Update()

    polydata = reader.GetOutput()

    points = polydata.GetPoints()
    array = points.GetData()
    numpy_nodes = vtk_to_numpy(array)
    #print(numpy_nodes)
    cells = polydata.GetPolys()
    nCells = cells.GetNumberOfCells()
    array = cells.GetData()
    # This holds true if all polys are of the same kind, e.g. triangles.
    assert(array.GetNumberOfValues()%nCells==0)
    nCols = array.GetNumberOfValues()//nCells
    numpy_cells = vtk_to_numpy(array)
    numpy_cells = numpy_cells.reshape((-1,nCols))
    return numpy_cells, numpy_nodes

def write_vtk(points, cell, filename):
    with open(filename, 'w') as f:
        # Write header
        f.write("# vtk DataFile Version 4.2\n")
        f.write("vtk output\n")
        f.write("ASCII\n")
        f.write("DATASET POLYDATA\n")
        
        # Write points
        num_points = len(points)
        f.write("POINTS {} float\n".format(num_points))
        for point in points:
            f.write("{} {} {}\n".format(point[0], point[1], point[2]))
        
        # Write cells
        num_cells = len(cell)
        total_num_points = np.sum(cell[:, 0])
        f.write("POLYGONS {} {}\n".format(num_cells, total_num_points + num_cells))
        for cell_indices in cell:
            num_indices = cell_indices[0]
            f.write("{} ".format(num_indices))
            for index in cell_indices[1:]:
                f.write("{} ".format(index))
            f.write("\n")

def visualize(nameOfFile):

        # Load the VTK file
    reader = vtk.vtkPolyDataReader()
    reader.SetFileName(nameOfFile)
    reader.Update()

    # Create a mapper
    mapper = vtk.vtkPolyDataMapper()
    mapper.SetInputConnection(reader.GetOutputPort())

    # Create an actor
    actor = vtk.vtkActor()
    actor.SetMapper(mapper)

    # Create a renderer and add the actor to it
    renderer = vtk.vtkRenderer()
    renderer.AddActor(actor)

    # Create a render window and set the renderer as its active renderer
    renderWindow = vtk.vtkRenderWindow()
    renderWindow.AddRenderer(renderer)
    renderWindow.SetSize(600, 600)
    renderer.ResetCamera()

    # Create an interactor and start the visualization
    interactor = vtk.vtkRenderWindowInteractor()
    interactor.SetRenderWindow(renderWindow)
    interactor.Start()

def flatten(Shape):
    return np.concatenate((Shape[:, 0], Shape[:, 1], Shape[:, 2]))

def unflatten(Shape):
    n = len(Shape)//3
    return np.array([Shape[:n], Shape[n:2*n], Shape[2*n:]]).T

def My_Visualize(Faces_List=[], Wiring=[], colors=[], Scatter=[0]):
    mesh = Faces_List[0][Wiring][:, 1:]
    offset = np.zeros(mesh.shape)
    vpl.figure("Face")
    for i in range(len(Faces_List)) :
        mesh = Faces_List[i][Wiring][:, 1:] + offset
        vpl.mesh_plot(mesh, color=colors[i])
        offset[:, :, 0] += 0.01
    if Scatter[0]:
        vpl.scatter(Faces_List[0][Scatter], radius=0.0001, color='r')
    vpl.show()

In [9]:
## Insert the path to faces files
path = '../FacesDatavtk/'

L = os.listdir(path)
Dataset = []
Wiring = []

for f in L :
    if 'vtk' in f :
       Tab, Tob = readfile(path+f)
       Dataset.append(Tob)
       Wiring.append(Tab)
Dataset = np.array(Dataset)

In [10]:
## Visualisation before the transformation

#for i in range(Dataset.shape[0]) :
#    My_Visualize([Dataset[i]], Wiring[0], ['w'], np.random.randint(0, len(Dataset[0]), 50))

In [11]:
mean_face = np.mean(Dataset,axis = 0)
My_Visualize([mean_face],  Wiring[0], ['w'])

In [12]:
Normalized_Faces = []
Path_w = '../Aligned_data/'

for i in range(Dataset.shape[0]):
    _, mtx, disparity = spatial.procrustes(mean_face, Dataset[i])
    Normalized_Faces.append(mtx)
    cell = Wiring[i]

    ## In case you want to visualize them to make sure of the alignment 
    #My_Visualize([mtx, Dataset[i]],  Wiring[0], ['w', 'w'])

    ## In case you want to save the aligned faces in vtk files 
    write_vtk(mtx, cell, Path_w+'Face'+str(i)+'.vtk')


## This table contains the aligned faces 
Normalized_Faces = np.array(Normalized_Faces)